In [16]:
from project.models import Project, Emprise
from public_data.models import Commune
from django.contrib.gis.db.models import Union

project = Project.objects.get(id=1542)
project

<Project: Diagnostic de De gascogne>

In [17]:
project.cities.all()

<QuerySet []>

In [18]:
combined_emprise = project.cities.all().aggregate(emprise=Union("mpoly"))
combined_emprise['emprise']

In [4]:
project.cities.all().values("name").distinct().count()
# .select_related("epci", "departement", "scot", "departement__region")

4246

In [5]:
project.cities.all().select_related("epci").values("epci__name").distinct().count()

164

In [6]:
project.cities.all().select_related("departement__region").values("departement__region__name").distinct().count()

1

In [8]:
qs = project.cities.all().select_related("epci", "departement", "scot", "departement__region")
qs.values("epci__name").distinct().count()

164

In [11]:
qs.values("departement__region__name").distinct().first()['departement__region__name']

'Occitanie'

In [12]:
qs.values("departement__name").distinct().first()['departement__name']

{'departement__name': 'Ariège'}

In [23]:
from metabase.models import StatDiagnostic

s = StatDiagnostic.objects.filter(project_id=1543).first()
s.project.name, s.created_date, s.region

('Diagnostic de CC du Saves',
 datetime.datetime(2023, 5, 4, 14, 2, 51, 8959, tzinfo=datetime.timezone.utc),
 None)

In [24]:
from metabase.tasks import async_create_stat_for_project
async_create_stat_for_project(project_id=1543, do_location=True)

[04052023 140728]ERROR   Cannot resolve keyword 'epci_name' into field. Choices are: artificialarea, communediff, communesol, departement, departement_id, epci, epci_id, id, insee, last_millesime, map_color, mpoly, name, pop, project, projectcommune, scot, scot_id, surface_artif
[04052023 140728]ERROR   Cannot resolve keyword 'epci_name' into field. Choices are: artificialarea, communediff, communesol, departement, departement_id, epci, epci_id, id, insee, last_millesime, map_color, mpoly, name, pop, project, projectcommune, scot, scot_id, surface_artif
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/django/db/models/sql/query.py", line 2114, in add_fields
    join_info = self.setup_joins(
  File "/usr/local/lib/python3.10/site-packages/django/db/models/sql/query.py", line 1776, in setup_joins
    path, final_field, targets, rest = self.names_to_path(
  File "/usr/local/lib/python3.10/site-packages/django/db/models/sql/query.py", line 1677, in names_t

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/django/db/models/sql/query.py", line 2114, in add_fields
    join_info = self.setup_joins(
  File "/usr/local/lib/python3.10/site-packages/django/db/models/sql/query.py", line 1776, in setup_joins
    path, final_field, targets, rest = self.names_to_path(
  File "/usr/local/lib/python3.10/site-packages/django/db/models/sql/query.py", line 1677, in names_to_path
    raise FieldError(
django.core.exceptions.FieldError: Cannot resolve keyword 'epci_name' into field. Choices are: artificialarea, communediff, communesol, departement, departement_id, epci, epci_id, id, insee, last_millesime, map_color, mpoly, name, pop, project, projectcommune, scot, scot_id, surface_artif

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_g

In [25]:
async_create_stat_for_project.delay(project_id=1543, do_location=True)

<AsyncResult: 2aa9e343-c274-4309-810f-3fe4e9d3970f>